collectData.py   
Johnathan Clementi   
March, 2022   

This script gathers data used to make the Hudson River School of Art mapper and visualizer webpage

In [1]:
from bs4 import BeautifulSoup
import urllib
import regex as re
import requests
import pandas as pd
import numpy as np
import json
import os

In [2]:
path = os.getcwd()

# The url to pull data from:
url = "https://en.wikipedia.org/wiki/List_of_Hudson_River_School_artists"

df = pd.read_html(url)[0]
df

,Artist Name,Portrait,Famous Work,Birth,Death,Description,Refs.
0,Charles Baker,NaN,More images,1839,1888,American landscape painter of the Hudson River...,NaN
1,William Bliss Baker,NaN,More images,27 November 1859,20 November 1886,American artist who began painting just as the...,[1]
2,John Dodgson Barrow,NaN,More images,24 November 1824,7 December 1906,NaN,NaN
3,Susie M. Barstow,NaN,More images,9 May 1836,12 June 1923,Avid hiker and early member of the Appalachian...,NaN
4,Julie Hart Beers,NaN,More images,1835,13 August 1913,One of the few women of the Hudson River Schoo...,NaN
...,...,...,...,...,...,...,...
66,Mary Josephine Walters,NaN,More images,1837,1883,NaN,NaN
67,Paul Weber,NaN,More images,19 January 1823,12 October 1916,NaN,NaN
68,Robert Walter Weir,NaN,More images,18 June 1803,1 May 1889,Elected to the National Academy of Design in 1...,NaN
69,Worthington Whittredge,NaN,More images,22 May 1820,25 February 1910,American artist of the Hudson River School. He...,NaN


In [17]:
# https://stackoverflow.com/questions/56757261/extract-href-using-pandas-read-html

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
table = soup.find('table')

links = []
for tr in table.findAll("tr"):
    trs = tr.find("td")
    for each in trs:
        try:
            link = each.find('a')['href']
            links.append(link)
        except:
            pass


In [ ]:
# empty list
data = []
list_header = []

header = soup.find_all("table")[0].find("tr")
  
for items in header:
    try:
        list_header.append(items.get_text())
    except:
        continue
  
# for getting the data 
HTML_data = soup.find_all("table")[0].find_all("tr")[1:]
  
for element in HTML_data:
    sub_data = []
    for sub_element in element:
        try:
            sub_data.append(sub_element.get_text())
        except:
            continue
    data.append(sub_data)
  
# Storing the data into Pandas
# DataFrame 
dataFrame = pd.DataFrame(data = data, columns = list_header)

In [3]:
# The url to pull data from:
url = "https://www.hudsonriverschool.org/hudsonrivervalley"

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
h3s = soup.findAll('h3')
h3s = np.asarray(h3s)

C:\Users\Johnathan\AppData\Local\Temp\ipykernel_9440\1010148947.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  h3s = np.asarray(h3s)


In [4]:
sites = []

for site in h3s:
  site = str(site)
  siteName = re.search("\;\"\>.*(\<br\/\>){1}", site)
  siteName = re.sub("^.*;\">", "", str(siteName))
  siteName = re.sub("<.*", "", str(siteName))
  if siteName != "None":
    sites.append(siteName)

sitesDF = pd.DataFrame(sites)
sitesDF.columns = ['SiteName']
# sitesDF

In [5]:
subpages = soup.findAll('div', attrs={"class":"intrinsic"})
# str(subpages[2])

pgLinks = []

for page in subpages:
  page = str(page)
  pageLink = re.search("href=\"\/.*\"", page)
  pageLink = re.sub("^.*match='href=\"","", str(pageLink))
  pageLink = re.sub("\"\'>", "", pageLink)
  pageLink = "https://www.hudsonriverschool.org{}".format(pageLink)
  pgLinks.append(pageLink)

for i in range(3):
  pgLinks.pop()

# pgLinks

sitesDF['pageLink'] = pgLinks


sitesDF.to_csv('{}\\sitenames.csv'.format(path))

In [6]:
siteCoords = []
long = []
lat = []

for url in pgLinks:
  response = requests.get(url)
  soup = BeautifulSoup(response.text, 'html.parser')
  ps = soup.findAll('p')

  for p in ps:
    pElement = str(p)
    coords = re.search("Site coordinates.*Long", pElement)
    latlong = re.sub("^.*match='","", str(coords))
    latlong = re.sub("\'>", "", latlong)
    if latlong != "None":
      siteCoords.append(latlong)

      lt = re.search("\: .*\, ", latlong)
      lt = re.sub("^.*match=\'\: ", "", str(lt))
      lt = re.sub(" Lat\.\, \'>", "", lt)
      lt = re.sub("\, \'>", "", lt)
      lat.append(lt)

      lng = re.search("\, .*Long", latlong)
      lng = re.sub("^.*match=\'\, ", "", str(lng))
      lng = re.sub(" Long\'>", "", lng)
      long.append(lng)

In [7]:
sitesLocDf = pd.DataFrame(siteCoords)
sitesLocDf['lat'] = lat
sitesLocDf['long'] = long

# Write to csv
sitesLocDf.to_csv('{}\\siteloc.csv'.format(path))

,0,lat,long
0,"Site coordinates: 42.22559 Lat., -73.861145 Long",42.22559,-73.861145
1,"Site coordinates: 42.217251 Lat., -73.827202 Long",42.217251,-73.827202
2,"Site coordinates: 42.223621 Lat., -73.887604 Long",42.223621,-73.887604
3,"Site coordinates: 42.184711 Lat., -74.074028 Long",42.184711,-74.074028
4,"Site coordinates: 42.19584, -74.06300 Long",42.19584,-74.06300
5,"Site coordinates: 42.200039 Lat., -74.041588 Long",42.200039,-74.041588
6,"Site coordinates: 42.205063 Lat., -74.030685 Long",42.205063,-74.030685
7,"Site coordinates: 42.194729 Lat., -74.034744 Long",42.194729,-74.034744
8,"Site coordinates: 42.256215 Lat., -73.797097 Long",42.256215,-73.797097
9,"Site coordinates: 41.922038 Lat., -73.980442 Long",41.922038,-73.980442


In [ ]:
imgs = soup.findAll('img')
imgs